<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/Transformer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Why?

- I'm curious of how transformer architecture look like under the hood.
- I wanted to know why things work and improve myself as Machine learning engineer.

## Input embeddings

In [2]:
# Create a simple vocabulary
tokens_to_ids = {
    "What": 1,
    "is": 2,
    "the": 3,
    "answer": 5,
    "of": 6,
    "life": 7,
    "42": 8,
    "<EOS>": 9
}

id_to_token = dict(map(reversed, tokens_to_ids.items()))

In [3]:
id_to_token
#

{1: 'What',
 2: 'is',
 3: 'the',
 5: 'answer',
 6: 'of',
 7: 'life',
 8: '42',
 9: '<EOS>'}